In [1]:
import os
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 https://developer.downlo

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar


--2022-10-07 22:19:15--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.60MB/s    in 0.2s    

2022-10-07 22:19:15 (5.60 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Pet_Products").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep='\t', header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [5]:
rows = df.count()
print(f"The number of records (rows) in the dataset is: {rows}")

The number of records (rows) in the dataset is: 2643619


In [6]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
df = df.drop("marketplace", "verified_purchase", "product_category", "review_headline", "review_body")

In [8]:
df.show()

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|          5|            0|          0|   N| 2015-08-31|
|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|          2|            0|          1|   N| 2015-08-31|
|   43214993|R14QJW3XF8QO1P|B0084OHUIO|     902215727|Tyson's True Chew...|          5|            0|          0|   N| 2015-08-31|
|   12835065|R2HB7AX0394ZGY|B001GS71K2|     568880110|Soft Side Pet Cra...|          5|            0|          0|   N| 2015-08-31|
|   26334022| RGKMPDQGSAHR3|B004ABH1LG|     692846826|EliteField 3-Door...|        

In [9]:
#fix column datatypes to match sql schema 
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType, IntegerType

df = df.withColumn("customer_id",col("customer_id").cast(IntegerType()))
df = df.withColumn("product_parent",col("product_parent").cast(IntegerType())) 
df = df.withColumn("review_date",col("review_date").cast(DateType()))
df = df.withColumn("customer_id",col("customer_id").cast(IntegerType())) 
df = df.withColumn("star_rating",col("star_rating").cast(IntegerType())) 
df = df.withColumn("helpful_votes",col("helpful_votes").cast(IntegerType())) 
df = df.withColumn("total_votes",col("total_votes").cast(IntegerType())) 


In [10]:
#check to make sure schema matches sql 
df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: date (nullable = true)



In [11]:
#drop nulls and check to make sure dataset is still sizeable 
df = df.dropna(how='any')
rows = df.count()
print(rows)

2643459


In [12]:
# Create dataframe to match review_id_table
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| REAKC26P07MDN|   28794885|B00Q0K9604|     510387886| 2015-08-31|
|R3NU7OMZ4HQIEG|   11488901|B00MBW5O9W|     912374672| 2015-08-31|
|R14QJW3XF8QO1P|   43214993|B0084OHUIO|     902215727| 2015-08-31|
|R2HB7AX0394ZGY|   12835065|B001GS71K2|     568880110| 2015-08-31|
| RGKMPDQGSAHR3|   26334022|B004ABH1LG|     692846826| 2015-08-31|
|R1DJCVPQGCV66E|   22283621|B00AX0LFM4|     590674141| 2015-08-31|
|R3V52EAWLPBFQG|   14469895|B00DQFZGZ0|     688538603| 2015-08-31|
|R3DKO8J1J28QBI|   50896354|B00DIRF9US|     742358789| 2015-08-31|
| R764DBXGRNECG|   18440567|B00JRCBFUG|     869798483| 2015-08-31|
| RW1853GAT0Z9F|   50502362|B000L3XYZ4|     501118658| 2015-08-31|
|R33GITXNUF1AD4|   33930128|B00BOEXWFG|     454737777| 2015-08-31|
|R1H7AVM81TAYRV|   43534290|B001HBBQKY|     420905252| 2015-08

In [13]:
# Create dataframe to match products table
products_table_df = df.select(["product_id", "product_title"])
products_table_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00Q0K9604|(8-Pack) EZwhelp ...|
|B00MBW5O9W|Warren Eckstein's...|
|B0084OHUIO|Tyson's True Chew...|
|B001GS71K2|Soft Side Pet Cra...|
|B004ABH1LG|EliteField 3-Door...|
|B00AX0LFM4|Carlson 68-Inch W...|
|B00DQFZGZ0|Dog Seat Cover Wi...|
|B00DIRF9US|The Bird Catcher ...|
|B00JRCBFUG|Cat Bed - Purrfec...|
|B000L3XYZ4|PetSafe Drinkwell...|
|B00BOEXWFG|Contech ZenDog Ca...|
|B001HBBQKY|Wellness Crunchy ...|
|B007O1FHB0|Rx Vitamins Essen...|
|B001P3NU30|Virbac C.E.T. Enz...|
|B00ZP6HS6S|Kitty Shack - 2 i...|
|B00IP05CUA|Wellness Kittles ...|
|B001U8Y598|OmniPet Anti-Ant ...|
|B011AY4JWO|K9KONNECTION [New...|
|B00DDSHE5A|SUNSEED COMPANY 3...|
|B00PJW5OR8|CXB1983(TM)Cute P...|
+----------+--------------------+
only showing top 20 rows



In [14]:
# Create dataframe to match customers table

customer_table_df = df.groupBy('customer_id').count()
customer_table_df = customer_table_df.withColumnRenamed('count','customer_count')
customer_table_df.show()


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   10270641|             1|
|   18365872|             1|
|   16711087|             1|
|   10742726|             2|
|   41169638|             1|
|   43622307|             1|
|   24540309|             2|
|   28258386|             1|
|   35329257|             2|
|   14552054|             1|
|   14529507|             5|
|   45392827|             5|
|   47282953|             1|
|    8201930|             1|
|   20109760|             2|
|   16405801|             4|
|   15056685|            21|
|   20840575|             2|
|   39048303|             1|
|    5596610|             1|
+-----------+--------------+
only showing top 20 rows



In [15]:
#change new customer_count column to integer type to match schema 
customer_table_df = customer_table_df.withColumn("customer_count",col("customer_count").cast(IntegerType()))
customer_table_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [16]:
# Create dataframe to match vine table
vine_table_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| REAKC26P07MDN|          5|            0|          0|   N|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|
|R14QJW3XF8QO1P|          5|            0|          0|   N|
|R2HB7AX0394ZGY|          5|            0|          0|   N|
| RGKMPDQGSAHR3|          5|            0|          0|   N|
|R1DJCVPQGCV66E|          5|            0|          0|   N|
|R3V52EAWLPBFQG|          3|            0|          0|   N|
|R3DKO8J1J28QBI|          2|            0|          0|   N|
| R764DBXGRNECG|          5|            1|          1|   N|
| RW1853GAT0Z9F|          5|            0|          0|   N|
|R33GITXNUF1AD4|          2|            0|          0|   N|
|R1H7AVM81TAYRV|          1|            2|          2|   N|
|R2ZOYAQZNNZZWV|          5|            0|          0|   N|
|R2FN1H3CGW6J8H|          1|            

In [22]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<amazon endpoint>:5432/postgres"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [18]:
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)
products_table_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)
customer_table_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)